In [ ]:
import pandas as pd
import numpy as np
import altair as alt

alt.renderers.enable('png')

# UWV Exploratory Analysis

Read classification information

According to the metadata, there are types of data on sick leave percentages:

- Accumulated over all economic activities
- Split into different sectors using SBI 2008
- Split on company size (1-10, 10-100 and 100+)



In [ ]:
classification = pd.read_csv('data/classificatie.csv', sep=';')
classification.info()

classification['Key'] = classification['Key'].astype('category');
classification.info()


In [ ]:
classification

So we need to make a tree out of this with T001081 at the top

```Tree
T001081
+> 301000
+> 300003
   +>305700
   +>307500



## Read data

The CSV file contains four columns (ID, BedrijfskenmerkenSBI2008, Perioden and Ziekteverzuimpercentage_1)
There are rows with only a spaces and a period which seem to indicate missing data.

In [ ]:
uwv = pd.read_csv('data/80072ned_UntypedDataSet_10052024_144733.csv', sep=';', na_values='       .')

uwv.info()


## Data fixing

Split the "Perioden" into three columns: Jaar (Year), RijType (is this row a quarterly row), Volgnummer (for quarterly rows, this indicates which quarter).

In [ ]:
uwv[['Jaar', 'VerzuimType', 'Kwartaalnummer']] = uwv['Perioden'].str.extract(r'(\d+)(KW|JJ)(\d+)', expand=True)

uwv['Jaar'] = uwv['Jaar'].astype(int)
uwv['Kwartaalnummer'] = uwv['Kwartaalnummer'].astype(int)
uwv['VerzuimType'] = uwv['VerzuimType'].astype('category')

uwv['Kwartaal'] = uwv.apply(lambda row: row['Jaar'] * 10 + row['Kwartaalnummer'], axis=1)

uwv['BedrijfskenmerkenSBI2008'] = uwv['BedrijfskenmerkenSBI2008'].astype('category')

uwv

## Get train and test

We will use 2022 and up as the final test data.
All prior tot 2022 will be training data. To test the trained model, we wil use 2021. So we get three splits:

- All data prior to 2021 is the real train data. This is the data to perform exploratory data analysis on. 
- All data from 2021 wil be the test set to test our trained models on. 
- When we are really done, 2022 and onwards will be the final test set. 

Additionally, we will only use the quarterly numbers (Verzuimtype = 'KW')

In [ ]:
uwv_test = uwv[(uwv['Jaar'] >= 2022) & (uwv['VerzuimType'] == 'KW')]
uwv_train = uwv[(uwv['Jaar'] < 2021) & (uwv['VerzuimType'] == 'KW')]
uwv_train_test = uwv[(uwv['Jaar'] == 2021) & (uwv['VerzuimType'] == 'KW')]

In [ ]:
uwv_train.info()

In [ ]:
uwv_train_2012plus = uwv_train[uwv_train['Jaar'] > 2012]

In [ ]:
alt.Chart(uwv_train_2012plus).mark_point().encode(
    x='Perioden', 
    y='Ziekteverzuimpercentage_1',
    color='BedrijfskenmerkenSBI2008'
)

In [ ]:
alt.Chart(uwv_train_2012plus).mark_boxplot().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
)

In [ ]:
alt.Chart(uwv_train_2012plus[uwv_train_2012plus['BedrijfskenmerkenSBI2008']=='T001081']).mark_line().encode(
    x='Kwartaalnummer',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
).properties(title='Seasonality of T001081 sick leave %')

In [ ]:
alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 1]).mark_point().encode(
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 2]).mark_point().encode(
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 3]).mark_point().encode(
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 4]).mark_point().encode(
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
)

In [ ]:

alt.Chart(uwv_train_2012plus[uwv_train_2012plus['BedrijfskenmerkenSBI2008']=='T001081']).mark_point().encode(
    x=alt.X(alt.repeat("column"), type='ordinal'),
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
).repeat(
    row=['Kwartaalnummer']
).properties(
    title='Seasonality of T001081 sick leave %'
)

In [ ]:
alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 1]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 2]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 3]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus[uwv_train_2012plus['Kwartaalnummer'] == 4]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
)

In [ ]:
uwv_train_2012plus_T = uwv_train_2012plus[uwv_train_2012plus['BedrijfskenmerkenSBI2008'] == 'T001081']
uwv_train_2012plus_T

In [ ]:

alt.Chart(uwv_train_2012plus_T[(uwv_train_2012plus_T['Kwartaalnummer'] == 1) ]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus_T[uwv_train_2012plus_T['Kwartaalnummer'] == 2]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus_T[uwv_train_2012plus_T['Kwartaalnummer'] == 3]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
) | alt.Chart(uwv_train_2012plus_T[uwv_train_2012plus_T['Kwartaalnummer'] == 4]).mark_point().encode(
    x='Perioden',
    y='Ziekteverzuimpercentage_1',
    color='Jaar'
)

In [ ]:
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20,10))

lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=1, ax=axes[0, 0])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=2, ax=axes[0, 1])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=3, ax=axes[0, 2])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=4, ax=axes[0, 3])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=5, ax=axes[1, 0])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=6, ax=axes[1, 1])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=7, ax=axes[1, 2])
lag_plot(uwv_train_2012plus_T['Ziekteverzuimpercentage_1'], lag=8, ax=axes[1, 3])

plt.show()

In [ ]:
import math

start_lag = 0
lag_length = 21

lagged_autocorrelation = pd.DataFrame()
lagged_autocorrelation['lag'] = range(start_lag, lag_length)

white_noise_border = 1.96 / math.sqrt(len(uwv_train_2012plus_T['Ziekteverzuimpercentage_1']))

wn_border = pd.DataFrame()
wn_border['lag'] = range(start_lag - 1, lag_length + 1)
wn_border['pos_white_noise_border'] = [white_noise_border for _ in range(start_lag - 1, lag_length + 1)]
wn_border['neg_white_noise_border'] = [-white_noise_border for _ in range(start_lag - 1, lag_length + 1)]

lagged_autocorrelation['autocorrelation'] = [uwv_train_2012plus_T['Ziekteverzuimpercentage_1'].autocorr(lag=lag) for lag in lagged_autocorrelation['lag']]

alt.Chart(lagged_autocorrelation).mark_bar().encode(
    x='lag',
    y='autocorrelation',
) + alt.Chart(wn_border).mark_line(strokeDash=[1,1]).encode(
    x='lag',
    y='pos_white_noise_border',
) + alt.Chart(wn_border).mark_line(strokeDash=[1,1]).encode(
    x='lag',
    y='neg_white_noise_border'
)

In [ ]:
moving_average = pd.DataFrame()

moving_average['quarter'] = uwv_train_2012plus_T['Perioden']
moving_average['sick'] = uwv_train_2012plus_T['Ziekteverzuimpercentage_1']

for window in range(3, 16, 2):
    moving_average[f'{window}-MA'] = uwv_train_2012plus_T['Ziekteverzuimpercentage_1'].rolling(window, center=True).mean()

moving_average


In [ ]:
charts = [alt.Chart(moving_average).mark_line().encode(x='quarter', y='sick')]

for window in range(3, 16, 2):
    charts.append(alt.Chart(moving_average).mark_line().encode(x='quarter', y=f'{window}-MA'))
    
alt.vconcat(*charts)

In [ ]:
from statsmodels.tsa.seasonal import STL

slp = uwv_train_2012plus_T['Ziekteverzuimpercentage_1']
slp.index = uwv_train_2012plus_T['Perioden']
slp

In [ ]:

plt.rc("font", size=6)
stl = STL(slp, period=4)
res = stl.fit()

fig = res.plot()
fig.autofmt_xdate()

